# Read Me

This project uses linkedin to send automatic messages in order to find a job faster

### Goals of the project
- Finding recruiters and people working in the industry i want and send them appropriate messages (adding the possibility to schedule a meeting with me on my personnal calendar ?
- Apply to the positions that fits my criteria (only working for simplified job apply imo)


### TODO
- Find optimal recruiter, find out how the localization filter works
- Work on finding optimal job post to send message to the recruiter from the job post


### pipeline to dm recruiters from job post
- get list of job throught the job search bar from linkedin (still can optimize this throught better keywords and location not working atm)
- iterate throught each job, if there is a recruiter who writed the job post, get it's id from selenium scrap
- send a connection invite proposing to have a quick chat about the role (add few keywords ?)
- connect the response to a collab excel to save the info


In [8]:
from linkedin_api import Linkedin
from linkedin_api.utils.helpers import get_id_from_urn

from my_functions.utils import *

from datetime import datetime

import openai

import selenium
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import unittest

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

import json

In [9]:
test_credential = get_credential('test_credential.txt')
true_credential = get_credential('linkedin_username.txt')

test_api = Linkedin(test_credential[0], test_credential[0])
real_api = Linkedin(true_credential[0], true_credential[0])



# First part : finding recruiters and sending them invite with personnal messages

- First dichotomy, from their summary we try to define if they are in the quant finance or in the tech sector to send the more accurate message

- Second one, if they are based in France we send our message in French

- To get better results we could developp a rating on how much we would be interested in each profile

# Currently outdated, they changed the search_people method


search_quant = test_api.search_people(keywords = 'quantitative finance', keyword_title = 'recruiter', regions = list(regions_known.values()), limit = 100)


connections = []
quant_recruiter = 0
data_recruiter = 0

from datetime import datetime
date_today = datetime.today().strftime('%Y-%m-%d')

for recruiter in search_quant:
        
    try:
        current_connection = add_gpt_personnalized_connection(recruiter, real_api)
        add_new_row(sheet, [recruiter['public_id'], current_connection[2], date_today, ""])
        print('no key error')
        
    except KeyError:    # Happends when no summary or profile can't be found
        print('key error')
        continue


## Work to send follow up messages after getting a new connection

In [15]:
conv = real_api.get_conversations()

In [4]:
conversations = real_api.get_conversations()

In [191]:
base_message = 'MSc'
follow_up_message = 'Thank you for connecting with me! \n I was wondering if you have a moment for a brief call. '\
                    + 'This would provide me with the chance to introduce myself and discuss the specific '\
                    + 'roles you are recruiting for. '\
                    + 'Additionally, it would allow us to explore whether there could be a potential match.'\
                    + '\nBest regards, \n Tanguy Bellec'

send_follow_up_message(real_api, base_message, follow_up_message, 150)

Hi Daniél, with my MSc in Statistics and experience as a data scientist, I believe you might have job opportunities that fit my qualifications. Looking forward to discussing further opportunities with you.
2-ODM0Y2E5YzktZjkzNy00NDk5LTk2NmQtMDMxNTQzYTU4MzU2XzAxMw==
Hello Tamryn I recently graduated with an MSc in Statistics and Quantitative Finance from Université PSL. As a passionate individual with an experience as a Data Scientist, I'm actively seeking a Quant position. I would love to discuss potential opportunities with you. 
2-MGMyNTY4ZWMtZTM0ZS00NjVlLWJiMjQtNWUwZWQzYmUzYTg1XzAxMg==
Hello David I recently graduated with an MSc in Statistics and Quantitative Finance from Université PSL. As a passionate individual with an experience as a Data Scientist, I'm actively seeking a Quant position. I would love to discuss potential opportunities with you. 
2-ZmQ3YmNlMDQtNzRhNC00NmUxLWExMWItOTIwMzAxMGRhZTdjXzAxMg==
Hello Zoe I recently graduated with an MSc in Statistics and Quantitative Fi

True

# Work to get optimzed messages

In [32]:
openai.api_key = open_file('open_ai_key.txt')[2:-2]

In [ ]:
mess_quant = "graduate from an MSc in Statistics and Quantitative Finance. An experience as a data scientist, looking for an opportunity as a quant"
mess_data = "graduate from an MSc in Statistics. An experience as a data scientist, looking for a new opportunity, data-driven."

In [ ]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant that personalize generic message asking for a connection on linkedin to headhunters from their summary."},
        {"role": "user", "content": f"Here is the generic message : <<{mess}>>"},
        {"role": "assistant", "content": "What is the summary from the headhunter?"},
        {"role": "user", "content": f"Here is the summary from his profile : <<{prof['summary']}>>"},
        {"role": "assistant", "content": "What is the expected output format?"},        
    ]
)

experience_prompt = "A motivation message in no more than 50 words to invite the headhunter to contact me if he sees a fit. Do it now."
experience, cv_discussion = utils.gpt3_chat(message=experience_prompt, messages=cv_discussion)
print(experience)

# Pipeline to dm recruiters from job post

https://www.linkedin.com/mynetwork/invitation-manager/sent/ link to see invite sent

In [54]:
DS_UK_jobs = test_api.search_jobs(keywords = 'Data Scientist', title = 'Data Scientist',
                                limit = 100, listed_at = 7 * 24 * 60 * 60, location_name = 'London, England')



In [46]:
test = test_api.search_jobs(keywords = 'Machine Learning',
                                limit = 1000, listed_at = 7 * 24 * 60 * 60, location_name = 'London, England')

In [57]:
browser = connect_firefox_to_my_linkedin(true_credential)
sheet = connect_to_sheet("messages_sent")

In [61]:
#browser = connect_firefox_to_my_linkedin(true_credential)

no_id = 0
jobs = test

for job in jobs[300:400]:
    
    current_job_id = job['entityUrn'].split(":")[3]
    recruiter_id = get_recruiter_id_from_job(browser, current_job_id)
    
    if(recruiter_id):
        
        job_title = job['title']
        job_title = gpt_extract_job_function_from_title(job_title)['job_title']
        
        message = create_message_for_recruiter_from_job(recruiter_id, job_title)
        
        connection = real_api.add_connection(recruiter_id, message)
        
        add_new_row(sheet, [current_job_id, recruiter_id, message])
    
        print(message)
    else:
        print("can't get id")
        no_id += 1
        
    time.sleep(3)
    
    
    
        

can't get id
Hello Augusta, 
I am a graduate from an Msc in Statistics with an experienceas a Data Scientist and I came across your job post for a Quantitative Researcher role. I am very interested in this opportunity and would love to learn more about the position.
can't get id
Hello Paul, 
I am a graduate from an Msc in Statistics with an experienceas a Data Scientist and I came across your job post for a Artificial Intelligence Engineer role. I am very interested in this opportunity and would love to learn more about the position.
can't get id
can't get id
can't get id
Hello Aliur, 
I am a graduate from an Msc in Statistics with an experienceas a Data Scientist and I came across your job post for a Analytics Consultant role. I am very interested in this opportunity and would love to learn more about the position.
Hello Jesse, 
I am a graduate from an Msc in Statistics with an experienceas a Data Scientist and I came across your job post for a Quantitative Researcher role. I am very 

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

# Finding jobs and easy apply to them

In [4]:
jobs = test_api.search_jobs(
        keywords='data scientist',
        companies=None,
        experience=None,
        job_type=None,
        job_title=None,
        industries=None,
        location_name='Paris, Île-de-France, France',
        remote=False,
        easyApply = True,
        listed_at=24 * 60 * 60,
        distance=None,
        limit=10,
        offset=0
    )